In [15]:
import numpy as np
from matplotlib import pyplot as plt

from dftpy.ions import Ions
from dftpy.functional import Functional
from dftpy.formats import io

from ase.io import read
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal

from scipy.optimize import minimize
import numpy as np
from matplotlib import pyplot as plt

from ase.io import read
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal

from scipy.optimize import minimize

from dftpy.ions import Ions
from dftpy.functional import Functional
from dftpy.formats import io
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional, ExternalPotential
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.optimization import Optimization

from ase.build import bulk


In [16]:
with open('he.out') as D:
    k = [match for match in D if "Kinetic energy" in match]
KS_ke = str(k).split()[6]


In [22]:
path_pp='/Users/valeria/Documents/PP/psl/'
file1='He.pz-rrkjus_psl.1.0.0.UPF'
PP_list = {'He': path_pp+file1}

In [24]:
XC = Functional(type='XC',name='LDA')
HARTREE = Functional(type='HARTREE')

In [17]:
def get_ions(path_file):
    inputfile = path_file
    ions = read(inputfile, format='vasp')
    return ions

In [18]:
material = Ions.from_ase(get_ions('he.vasp'))

In [32]:
def min_delta_rho(x0, *args):
    rho_ks, ions = args
    KE = Functional(type='KEDF',name='WT', rho0=x0, sigma=0.02)
#     ions.set_charges(4)
    grid = DirectGrid(lattice=ions.cell, nr=[len(rho_ks),len(rho_ks),len(rho_ks[0,0])])
    rho_ini = DirectField(grid=grid)
    rho_ini[:] = ions.get_ncharges()/ions.cell.volume 
    PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list, rcut=10)
    evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)
    optimization_options = {'econv' : 1e-5*ions.nat}
    opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options, 
                       optimization_method = 'CG')
    rho = opt.optimize_rho(guess_rho=rho_ini)
    rho_ini[:]=rho
    diff = 0.5 * (np.abs(rho - rho_ks)).integral()
    print(diff)
    return diff

In [33]:
def min_energy(x0, *args):
        KS_KE, rho = args
        KE = Functional(type='KEDF',name='WT', rho0=x0)
        OF_KE = KE(rho).energy
        diff = np.abs(KS_KE*1/2-OF_KE)
        return diff

In [39]:
rho_ks = io.read_density('he.xsf')
# KS_KE = float(KS_ke)
minn = minimize(min_delta_rho, 1e-9, args = (rho_ks, material), method='L-BFGS-B', bounds=[[0,5]], options={'ftol' : 1e-4})

setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       3.151197E-01    
1       -2.061487867697E+00     -1.649472E+00   1.707753E+00    1       5       1.032588E+00    
2       -2.156085476765E+00     -9.459761E-02   3.810642E+00    1       3       1.475331E+00    
3       -2.191140920429E+00     -3.505544E-02   2.331116E+00    1       2       1.769160E+00    
4       -2.234840486539E+00     -4.369957E-02   4.787020E+00    1       2       2.063967E+00    
5       -2.295463311922E+00     -6.062283E-02   6.132236E+00    1       2       2.366319E+00    
6       -2.362275890876E+00     -6.681258E-02   4.481997E+00    1       1       2.519347E+00    
7       -2.420565940429E+00     -5.829005E-02   7.453345E+00    1       1       2.676324E+00    
8       -2.487447362198E+00     -6.688142E-02   5

KeyboardInterrupt: 

In [35]:
minn.x, minn.fun

(array([0.1]), 0.13291427651937252)

In [43]:
min_delta_rho(1e-7,rho_ks, material)

setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       2.781250E-01    
1       -2.051291280708E+00     -1.639275E+00   1.703231E+00    1       5       1.021752E+00    
2       -2.146108019446E+00     -9.481674E-02   3.771227E+00    1       3       1.477575E+00    
3       -2.180458400005E+00     -3.435038E-02   2.356123E+00    1       2       1.775319E+00    
4       -2.223575041938E+00     -4.311664E-02   4.746175E+00    1       2       2.073843E+00    
5       -2.283835132103E+00     -6.026009E-02   5.867163E+00    1       2       2.366424E+00    
6       -2.350466037311E+00     -6.663091E-02   4.330892E+00    1       1       2.517509E+00    
7       -2.408008619147E+00     -5.754258E-02   7.260004E+00    1       1       2.664923E+00    
8       -2.477189240804E+00     -6.918062E-02   7

0.06668380769835179

In [44]:
min_delta_rho(1e-6,rho_ks, material)

setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       2.899361E-01    
1       -2.051585175000E+00     -1.639569E+00   1.703426E+00    1       5       1.035048E+00    
2       -2.146230364783E+00     -9.464519E-02   3.785036E+00    1       3       1.495302E+00    
3       -2.180702126590E+00     -3.447176E-02   2.345166E+00    1       2       1.794352E+00    
4       -2.223747455118E+00     -4.304533E-02   4.730542E+00    1       2       2.086736E+00    
5       -2.283994693995E+00     -6.024724E-02   5.866638E+00    1       2       2.372662E+00    
6       -2.350643340018E+00     -6.664865E-02   4.331723E+00    1       1       2.520246E+00    
7       -2.408410483841E+00     -5.776714E-02   7.227253E+00    1       1       2.669121E+00    
8       -2.474309628460E+00     -6.589914E-02   5

0.0656723813760336

In [38]:
min_delta_rho(1.5,rho_ks, material)

setting key: He -> /Users/valeria/Documents/PP/psl/He.pz-rrkjus_psl.1.0.0.UPF
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       -4.120160008539E-01     -4.120160E-01   3.435016E-02    1       1       2.722063E-01    
1       -1.319403118928E+00     -9.073871E-01   1.186916E+00    1       5       9.918082E-01    
2       -1.378567123274E+00     -5.916400E-02   1.071358E+00    1       2       1.280318E+00    
3       -1.390269647272E+00     -1.170252E-02   9.359919E-01    1       2       1.586248E+00    
4       -1.401977425051E+00     -1.170778E-02   1.013599E+00    1       2       1.883709E+00    
5       -1.411464998548E+00     -9.487573E-03   8.856484E-01    1       1       2.035438E+00    
6       -1.417379606968E+00     -5.914608E-03   4.289564E-01    1       2       2.323268E+00    
7       -1.420980321817E+00     -3.600715E-03   2.857031E-01    1       2       2.616696E+00    
8       -1.424486635146E+00     -3.506313E-03   2

83      -1.449173206699E+00     -8.152102E-06   6.696181E-04    1       1       1.729320E+01    
84      -1.449180473258E+00     -7.266559E-06   5.593856E-04    1       2       1.758648E+01    
85      -1.449187182238E+00     -6.708980E-06   5.805271E-04    1       1       1.773918E+01    
86      -1.449194130899E+00     -6.948661E-06   4.661025E-04    1       1       1.789392E+01    
87      -1.449200117425E+00     -5.986526E-06   5.522459E-04    1       1       1.804509E+01    
88      -1.449207004368E+00     -6.886942E-06   7.134265E-04    1       2       1.834890E+01    
89      -1.449214589093E+00     -7.584725E-06   5.228220E-04    1       1       1.849468E+01    
90      -1.449220755854E+00     -6.166761E-06   4.982546E-04    1       2       1.878483E+01    
91      -1.449226675539E+00     -5.919685E-06   4.844375E-04    1       1       1.893343E+01    
92      -1.449233170218E+00     -6.494679E-06   3.978260E-04    1       1       1.907958E+01    
93      -1.449238615155E+00   

0.8360087867127698

In [16]:
np.save('rho_DEN', minn.x)